In [1]:
import csv
import google.auth
import json
import sys

from google.api_core.exceptions import AlreadyExists
from google.cloud import pubsub_v1
from google.cloud.pubsub_v1.types import BatchSettings

from pathlib import Path
from typing import Any

In [ ]:
_, project_id = google.auth.default()
print(project_id)

In [ ]:
topic_id = "aula-pdm-anuncios"
# Dentro do diretório anuncions, existem vários arquivos distintos
# Cada arquivo contém anúncios de um estado ou cidade específica
# Você pode escolher qualquer um deles para publicar
anuncios_file = "/home/dataproc/aula-pdm-pubsub/dados/anuncios/imoveis_go-goiania.jsonl"

In [ ]:
batch_settings = BatchSettings(
    max_bytes=5 * 1024 * 1024,  # No máximo 5MB por lote
    max_messages=1000,  # até 1000 messages por lote
    max_latency=0.25,  # envia a cada 250ms
)
publisher = pubsub_v1.PublisherClient(batch_settings=batch_settings)

In [ ]:
topic_path = publisher.topic_path(project_id, topic_id)
try:
    publisher.create_topic(name=topic_path)
except AlreadyExists:
    print(f"O tópico {topic_id} já existe.")

In [3]:
# Lê o JSONL e armazena os JSONs convertidos em uma lista
linhas_json = Path(anuncios_file).open("r", encoding="utf-8").readlines()
num_rows = len(linhas_json)

print(f"Foram lidas {num_rows} linhas do arquivo {anuncios_file}.")

In [ ]:
print(linhas_json[0])

In [ ]:
futures = []
for idx, linha in enumerate(linhas_json):
    future = publisher.publish(topic_path, data=linha.encode("utf-8"), row=str(idx))
    futures.append(future)

In [ ]:
for fut in futures:
    fut.result(timeout=60)